In [ ]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import SystemMessage, HumanMessage
from typing import Annotated, Optional, TypedDict
import pytesseract
from PIL import Image

# -------------------
# Tools
# -------------------

def divide(a: int, b: int) -> float:
    """Simple calculator tool."""
    return a / b

def extract_text(img_path: str) -> str:
    """Extract text from image using pytesseract OCR."""
    try:
        img = Image.open(img_path)
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        return f"Error reading image: {e}"

# -------------------
# Agent State
# -------------------

class AgentState(TypedDict):
    input_file: Optional[str]
    messages: Annotated[list, add_messages]

# -------------------
# Models
# -------------------

# Local model: change if you have another Ollama model
llm = ChatOllama(model="qwen2.5-coder:7b")

# Bind tools (enables ReAct pattern)
# llm_with_tools = llm.bind_tools([divide, extract_text]), parallel_tool_calls=False)
llm_with_tools = llm.bind_tools([divide, extract_text])

# -------------------
# Assistant Node
# -------------------

def assistant(state: AgentState):
    """
    Alfred the butler can analyze documents, call tools,
    and reason over the extracted text.
    """
    sys_msg = SystemMessage(
        content=(
            "You are Alfred, a helpful butler who can analyze documents and call tools. "
            "When extract_text returns text, read it carefully as if it were a document. "
            "Then, follow the human’s request based on that extracted content. "
            "For example, if asked to pick items, read the extracted text and select from it."
        )
    )
    return {
        "messages": [llm_with_tools.invoke([sys_msg] + state["messages"])],
        "input_file": state["input_file"],
    }


#testing the text extract
text = extract_text("menu.jpg")
print("================================ Debug Start =================================")
print("extract_text")
print(text)
print("================================ Debug End   =================================")


# -------------------
# Graph Setup
# -------------------

builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([divide, extract_text]))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# -------------------
# Run Query
# -------------------

messages = [
    HumanMessage(
        content="From the provided image menu.jpeg, pick any three items you recommend."
    )
]

result = react_graph.invoke({"messages": messages, "input_file": "menu.jpeg"})

# -------------------
# Display Messages
# -------------------

for m in result["messages"]:
    m.pretty_print()
